Note: following these instructions requires permissions to sudo as `analytics-product`, which owns the `canonical_data` database.

In [1]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd

import wmfdata as wmf

sys.path.insert(0, "..")
from utils import assert_tsv_loaded_correctly

Start by loading the dataset into your own database.

In [2]:
MY_DATABASE = "nshahquinn"

wmf.hive.load_csv(
    "wikis.tsv",
    field_spec="""
        database_code   STRING  COMMENT 'Same as wiki_db in MediaWiki History, e.g. enwiki', 
        domain_name     STRING  COMMENT 'e.g. en.wikipedia.org', 
        database_group  STRING  COMMENT 'e.g. wikipedia',
        language_code   STRING  COMMENT 'e.g. en',
        mobile_domain_name STRING COMMENT 'e.g. en.m.wikipedia.org',
        language_name   STRING  COMMENT 'e.g. English',
        status          STRING  COMMENT 'open/closed',
        visibility      STRING  COMMENT 'public/private',
        editability     STRING  COMMENT 'public/private',
        english_name    STRING  COMMENT 'e.g. English Wikipedia'
    """,
    db_name=MY_DATABASE,
    table_name="wikis",
    sep="\t"
)

/srv/home/nshahquinn-wmf/wmfdata-python/wmfdata/hive.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  response = pd.read_sql(command, conn)


Now, since it's not possible to run `sudo` commands in our Jupyter environment, in a plain SSH connection, run the following command:

In [3]:
print(
    "sudo -u analytics-product kerberos-run-command analytics-product "
    "hive -e 'DROP TABLE IF EXISTS canonical_data.wikis; "
    f"CREATE TABLE canonical_data.wikis AS SELECT * FROM {MY_DATABASE}.wikis'"
)

sudo -u analytics-product kerberos-run-command analytics-product hive -e 'DROP TABLE IF EXISTS canonical_data.wikis; CREATE TABLE canonical_data.wikis AS SELECT * FROM nshahquinn.wikis'


Once that has completed, verify that the newly loaded data matches our local copy.

In [4]:
assert_tsv_loaded_correctly("wikis.tsv", "canonical_data.wikis")